In [7]:
! pip install cohere umap-learn altair annoy datasets tqdm

In [8]:
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [22]:
# Load the dataset answers.csv
df = pd.read_csv('answers.csv')
df.head(10)

,text,coarse_label,fine_label
0,Photosynthesis is the process by which plants use sunlight water and carbon dioxide to create oxygen and energy in the form of sugar,1.0,1.0
1,Photosynthesis is a process by which phototrophs convert light energy into chemical energy which is later used to fuel cellular activities. The chemical energy is stored in the form of sugars which are created from water and carbon dioxide,1.0,1.0
2,Photosynthesis is the process used by plants algae and some bacteria to turn sunlight into energy. The process chemically converts carbon dioxide (CO2) and water into food (sugars) and oxygen. The chemical reaction often relies on a pigment called chlorophyll which gives plants their green color,1.0,1.0
3,Plants use sunlight water and the gases in the air to make glucose which is a form of sugar that plants need to survive. This process is called photosynthesis and is performed by all plants algae and even some microorganisms. To perform photosynthesis plants need three things: carbon dioxide water and sunlight,1.0,1.0
4,Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that through cellular respiration can later be released to fuel the organism's activities,1.0,1.0
5,Photosynthesis the process by which green plants and certain other organisms transform light energy into chemical energy. During photosynthesis in green plants light energy is captured and used to convert water carbon dioxide and minerals into oxygen and energy-rich organic compounds.,NaN,NaN
6,Photosynthesis is the process in which light energy is converted to chemical energy in the form of sugars. In a process driven by light energy glucose molecules (or other sugars) are constructed from water and carbon dioxide and oxygen is released as a byproduct,1.0,1.0
7,Green plants make sugar for growth by a process called photosynthesis which means making things with light. Plants use sunlight water and carbon dioxide to create oxygen and energy in the form of sugar,1.0,1.0
8,Photosynthesis is a chemical process by which plants some bacteria and algae convert energy derived from sunlight to chemical energy. This is an important process for biological life on earth because it allows energy from sunlight to be harnessed and transferred into a form that can be utilized by organisms to fuel their activity,1.0,1.0


In [16]:
# export the data to a csv file
# df.to_csv('trec.csv', index=False)

In [23]:
# Paste your API key here. Remember to not share publicly
api_key = ''

# Create and retrieve a Cohere API key from dashboard.cohere.ai/welcome/register
co = cohere.Client(api_key)

# Get the embeddings
embeds = co.embed(texts=list(df['text']),
                  model='large',
                  truncate='LEFT').embeddings

In [24]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(len(embeds[0]), 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])
search_index.build(10) # 10 trees
search_index.save('test.ann')

True

In [38]:
# Choose an example (we'll retrieve others similar to it)
# example_id = 8
# Retrieve nearest neighbors
# similar_item_ids = search_index.get_nns_by_item(example_id,10,
#                                                 include_distances=True)

#get by user input
user_input = input("Enter the answer for the question: ")
user_input = [user_input]
user_input_embeds = co.embed(texts=user_input,
                    model='large',
                    truncate='LEFT').embeddings
similar_item_ids = search_index.get_nns_by_vector(user_input_embeds[0],10,
                                                include_distances=True)

# Format and print the text and distances
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'], 
                             'label': df.iloc[similar_item_ids[0]]['coarse_label'],
                             'distance': similar_item_ids[1]})
print(f"Question:'{user_input}'\nNearest neighbors:")
results

Question:'['photosynthesis is when plants use c02 and sunlight to create energy that is to be used by the plat. In addition to this water is also created.']'
Nearest neighbors:


,texts,label,distance
0,Photosynthesis is the process by which plants use sunlight water and carbon dioxide to create oxygen and energy in the form of sugar,1.0,0.513795
4,Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that through cellular respiration can later be released to fuel the organism's activities,1.0,0.668759
5,Photosynthesis the process by which green plants and certain other organisms transform light energy into chemical energy. During photosynthesis in green plants light energy is captured and used to convert water carbon dioxide and minerals into oxygen and energy-rich organic compounds.,NaN,0.688115
2,Photosynthesis is the process used by plants algae and some bacteria to turn sunlight into energy. The process chemically converts carbon dioxide (CO2) and water into food (sugars) and oxygen. The chemical reaction often relies on a pigment called chlorophyll which gives plants their green color,1.0,0.689623
1,Photosynthesis is a process by which phototrophs convert light energy into chemical energy which is later used to fuel cellular activities. The chemical energy is stored in the form of sugars which are created from water and carbon dioxide,1.0,0.738453
6,Photosynthesis is the process in which light energy is converted to chemical energy in the form of sugars. In a process driven by light energy glucose molecules (or other sugars) are constructed from water and carbon dioxide and oxygen is released as a byproduct,1.0,0.740152
3,Plants use sunlight water and the gases in the air to make glucose which is a form of sugar that plants need to survive. This process is called photosynthesis and is performed by all plants algae and even some microorganisms. To perform photosynthesis plants need three things: carbon dioxide water and sunlight,1.0,0.775085
8,Photosynthesis is a chemical process by which plants some bacteria and algae convert energy derived from sunlight to chemical energy. This is an important process for biological life on earth because it allows energy from sunlight to be harnessed and transferred into a form that can be utilized by organisms to fuel their activity,1.0,0.778387
7,Green plants make sugar for growth by a process called photosynthesis which means making things with light. Plants use sunlight water and carbon dioxide to create oxygen and energy in the form of sugar,1.0,0.807096


In [40]:
# 1 is the question category
mean_distance = results[results['label'] == 1]['distance'].mean()
if mean_distance < 1:
    print("The answer is correct")
else:
    print("The answer is incorrect")

The answer is correct
